Nezbytné importy.

Funkce `kdf (password, salt = b'', rounds = 1)` bude sloužit k zahashování hesla, jenž se bude předávat prvním parametrem `password`. Volitelně lze ještě zadat sůl a počet iterací.

In [42]:
import time
import hashlib
import sys
import itertools

# Výpočet hash z hesla s použitím soli a zvoleného počtu iterací
def kdf (password: bytes, salt: bytes = b'', rounds: int = 1) -> bytes:
  h = salt + password
  for i in range(rounds):
    dgst = hashlib.sha512()
    dgst.update(h)
    h = dgst.digest()
  return h # Výsledná hash

In [43]:
# Vyzkoušíme funkci kdf pro výpočet hashe hesla
print(kdf(b'AAA')) # bytes
print(kdf(b'AAA').hex()) # hex string

b'\x8dp\x8d\x18\xb5M\xf3\x96-io\x06\x9a\xd4-\xadwb\xb5\xd4\xd3\xc9~\xe5\xfa-\xae\x06s\xedFTQd\xc0x\xb8\xdb=Y\xc4\xb9` \xe41o\x17\xbb=\x91\xbf\x1fk\xc0\x89k\xbeuAn\xb8\xc3\x85'
8d708d18b54df3962d696f069ad42dad7762b5d4d3c97ee5fa2dae0673ed46545164c078b8db3d59c4b96020e4316f17bb3d91bf1f6bc0896bbe75416eb8c385


V souboru `English.dic` je seznam slov, která nám budou sloužit pro slovníkový útok, tzn. budeme uvažovat hesla pouze z tohoto seznamu. Pro následné rychlé získání hesla z hashe si předpočítáme hashe pro všechna slova v tomto seznamu.



In [44]:
# Načtení slov ze souboru
with open("English.dic", "r") as fin:
  passwords=fin.readlines()

passwords = [x.strip() for x in passwords] # Odstranění newline

In [45]:
# Vytvoříme slovník (dictionary - dále v textu budeme používat také "dict" pro rozlišení datového typu v Pythonu)
# - budou zde uloženy dvojice klíč:hodnota (hash:heslo v našem případě), indexuje se pomocí klíče
hashPasswordTable = {kdf(bytes(x, 'utf8')):x for x in passwords}

In [46]:
# Příklad použití slovníku: Zachytili jsme následující hash, a víme, že heslo je ze slovníku.
testHash = bytes.fromhex("8d708d18b54df3962d696f069ad42dad7762b5d4d3c97ee5fa2dae0673ed46545164c078b8db3d59c4b96020e4316f17bb3d91bf1f6bc0896bbe75416eb8c385")
# Jaké bylo odpovídající původní heslo?
print(hashPasswordTable[testHash])

AAA


## Úkol 1
Jak bude velká tato tabulka? Použitá hashovací funkce je SHA512.
Stačí řádově, neuvažujte reprezentaci slovníku (datového typu dict) v Pythonu.

In [47]:
all_passwords = [len(x.strip()) for x in passwords]
table_size = len(passwords) * (64 + (sum(all_passwords) / len(passwords)))
print(f'Table size:{table_size / 1024 ** 2: .2f} MB')

Table size: 22.30 MB


**Odpověď:**
cca 22.3 MB

Nyní můžeme pro srovnání zkusit rychlost nalezení hesla při použití hrubé síly (zkoušení všech možných kombinací povolených znaků hesla), nebo předpočítané tabulky.

**Poznámka:** Je dobré si uvědomit, že předpočítaná tabulka je pouze ze slov ve slovníku English.dic, tzn. je už sama o sobě značně omezená. Pokud bychom předpočítávali hash pro všechna možná hesla do určité délky, tabulka by byla značně větší.

In [48]:
target = kdf(b'phial')

In [49]:
%%time
for x in itertools.product('abcdefghijklmnopqrstuvwxyz', repeat=5): # procházení všech kombinací malých písmen o délce 5
  p = ''.join(x)
  if kdf(bytes(p, 'ascii')) == target:
    print(p)
    break

phial
CPU times: user 8.7 s, sys: 0 ns, total: 8.7 s
Wall time: 8.7 s


In [50]:
%time
hashPasswordTable[target]

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.62 µs


'phial'

## Úkol 2
Pro zamezení nebo ztížení využití předpočítaných tabulek se využívá sůl.

Prostá hash hesla: hash = H (password)

Osolená hash: salted_hash = H (salt || password)

Uvažujme zjednodušený případ, kdy sůl může být pouze jeden malý znak (a-z).
Vytvořte novou předpočítanou tabulku, která bude obsahovat všechny možné kombinace soli a slov ze seznamu výše. (Pro naše účely se sůl jednoduše zřetězí se slovem před zahashováním, lze ji také zadat jako argument volání funkce `kdf`)

-------------------

Jak bude velká tato tabulka?

In [51]:
saltedhashPasswordTable = {}

saltedhashPasswordTable = {kdf(bytes(password, 'utf-8'), bytes(chr(salt), 'utf-8')): password
                           for salt in range(ord('a'), ord('z') + 1)
                           for password in passwords}

In [57]:
print(f'Salted table size:{table_size * 26 / 1024 ** 2: .2f} MB')

Salted table size: 579.91 MB


**Odpověď:**
cca 579.91 MB


Jak dlouhá by měla být sůl v případě, že bychom chtěli, aby výsledná předpočítaná tabulka byla větší než 1TB? Předpokládejte stále stejný slovník, do velikosti tabulky pro jednoduchost stačí uvažovat pouze velikost 1 hash a její počet (nemusíte zakomponovávat velikost řetězců reprezentující heslo a sůl).
U délky soli uvažujte 2 případy:
1) Sůl sestávající z malých znaků (a-z)

In [61]:
salt_length = 0
while(True):
    # 26^salt_length different possibilities of salt
    if (26 ** salt_length) * table_size / 1024 ** 4 > 1:
        break
    salt_length = salt_length + 1
print(f'Salt should contain {salt_length} characters.')

Salt should contain 4 characters.


2) Sůl sestávající z libovolných bytů (hexadecimálně 0x00-0xFF)
(Může být výhodné si nejprve vyjádřit, jakou entropii by sůl měla mít.)

In [63]:
salt_length = 0
while(True):
    # 256^salt_length different possibilities of salt
    if (256 ** salt_length) * table_size / 1024 ** 4 > 1:
        break
    salt_length = salt_length + 1
print(f'Salt should contain {salt_length} characters.')

Salt should contain 2 characters.


**Odpověď:** 
1. Delka 4 znaky.
2. Delka 2 znaky.

-------------------
Poté si zvolte náhodně sůl (1 znak) a 1 slovo ze seznamu, které poslouží jako vaše heslo. Tuto kombinaci zahashujte, vzájemně si pošlete ve dvojicích a zjistěte heslo vašeho kolegy.

**Poznámka:** Kromě samotné hashe můžete kolegovi prozradit i sůl. V běžném scénáři (únik databáze) jsou k dispozici všechny údaje nutné pro výpočet hashe (použitý algoritmus, sůl, počet iterací), kromě samotného hesla.

Kromě soli se pro ztížení útoků využívá také vyšší počet iterací vybrané hashovací funkce.

In [64]:
%%time
kdf(b'abcdefgh', rounds=10000)

CPU times: user 7.6 ms, sys: 0 ns, total: 7.6 ms
Wall time: 7.51 ms


b'\xbb,;(!\x8eb\xc9\x9a \xaa\xdfS\x8b\xee\xe0\xcbsKR\x9aT\xfa\xd3d\x8c?\xf2\x81\xfd\xe9\x8e*\xfd[uG\x9dM\xb4>e\xaeP\xd6\x9f$\xad\xaf\xc2\xf3/ \xc8m\xbdG\xf7]\xa1\x08\xa4t\xc4'

## Úkol 3

Spočtěte váš hash rate (počet hashů za vteřinu, které dokážete spočítat).

In [77]:
start = time.time()
hash_rate = 0
while time.time() - start <= 1:
    kdf(b"In the process of earning two points.....")
    hash_rate = hash_rate + 1
print(f'Hash rate: {hash_rate}')

Hash rate: 952208


**Odpověď:**  952208

Kolik iterací hashovací fce bude potřeba nastavit při tvorbě hashe z hesla, aby útočníkovi trvalo přibližně rok jeho prolomení? Předpokládejme, že heslo je vytvořeno zcela náhodně z tisknutelných ASCII znaků (95), je dlouhé 8 znaků, hash rate útočníka je 1000000000000 hash/vteřina (1 terahash/s). Jak dlouho by v takovém případě trval výpočet hash z hesla na vašem zařízení?

In [82]:
combinations_of_passwords = 95 ** 8
striker_hash_rate = 10 ** 12
year_in_seconds = 365 * 24 * 60 * 60

hashes_per_year = year_in_seconds * striker_hash_rate
iteration_number = hashes_per_year / combinations_of_passwords
print(f'Iteration number: {iteration_number:.0f}')

Iteration number: 4754


In [87]:
%%time
kdf(b'8charsIn', rounds=4754)

CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 2.87 ms


b'\xf2\xb2<\xe2g2\xbc>Jr\xffck\x83B\xbcT\x17\xcb\xcc\xd0\xd6\x9d=\xf49\xfd{\x82x\x1c\x81\x03\xe5F\'\xb6B+\x85\x87"B;2>\x9c\x99d\xa1ha\x87\x8e\xc5\xea\n5\xdc\x8d\xcc51\xaf'

**Odpověď:**
* Number of iteration: 4754
* My time: CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms, 
Wall time: 2.87 ms